In [1]:
import heeps
import numpy as np
from astropy.io import fits

In [2]:
def get_psfs(conf):
    conf.update(
        mode = 'ELT',
        f_phase = 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_scao_only_L_285.fits',
        avg = True,
        hfov = 1.065,
        ngrid = 2048,
    )
    conf = heeps.config.read_config(**conf)
    conf['band_specs']['L']['pscale'] = 5.47/2
    conf = heeps.config.update_config(**conf)
    wf = heeps.pupil.pupil(**conf)
    psf = heeps.wavefront.propagate(wf, **conf)
    elt_peak = np.max(psf)
    print('ELT peak = %.4f'%elt_peak)
    psf /= elt_peak
    fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_ELT.fits', psf, overwrite=True)
    
    verbose = False
    for conf['mode'] in ['RAVC', 'APPIMG', 'CVC', 'CLC']:
        if 'RAVC' in conf['mode']:
            conf.update(ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
        elif 'CVC' in conf['mode']:
            conf.update(ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
        elif 'CLC' in conf['mode']:
            conf.update(ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)
        conf = heeps.config.update_config(**conf, verbose=verbose)
        wf = heeps.pupil.pupil(**conf)
        if 'VC' in conf['mode']:
            psf_OFF = heeps.wavefront.propagate(wf, onaxis=False, **conf)
            psf_ON = heeps.wavefront.propagate(wf, onaxis=True, **conf)
            print('mode = %s, leakage = %.2e'%(conf['mode'], np.sum(psf_ON)/np.sum(psf_OFF)))
        else:
            psf_ON = heeps.wavefront.propagate(wf, onaxis=True, **conf)
        psf_ON /= elt_peak
        fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_%s.fits'%conf['mode'], psf_ON, overwrite=True)

### ideal

In [3]:
conf = dict(
    dir_output = 'instant_psf/ideal',
    nframes_avg = 1,
    add_phase = False,
)
get_psfs(conf)

   2022-06-29 16:53:45, e2e simulation using 56 cores
   2022-06-29 16:53:50, completed in 4.84 seconds
ELT peak = 0.0122
   2022-06-29 16:53:52, e2e simulation using 56 cores
   2022-06-29 16:53:58, completed in 6.06 seconds
   2022-06-29 16:53:59, e2e simulation using 56 cores
   2022-06-29 16:54:10, completed in 10.67 seconds
mode = RAVC, leakage = 2.15e-03
   2022-06-29 16:54:11, e2e simulation using 56 cores
   2022-06-29 16:54:16, completed in 4.96 seconds
   2022-06-29 16:54:18, e2e simulation using 56 cores
   2022-06-29 16:54:23, completed in 4.73 seconds
   2022-06-29 16:54:24, e2e simulation using 56 cores
   2022-06-29 16:54:35, completed in 10.64 seconds
mode = CVC, leakage = 2.59e-02
   2022-06-29 16:54:37, e2e simulation using 56 cores
   2022-06-29 16:54:46, completed in 9.93 seconds


### scao only

In [4]:
conf = dict(
    dir_output = 'instant_psf/scao_only',
    nframes_avg = 1,
    add_phase = True,
)
get_psfs(conf)

   2022-06-29 16:54:48, e2e simulation using 56 cores
   2022-06-29 16:54:52, completed in 4.26 seconds
ELT peak = 0.0118
   2022-06-29 16:54:55, e2e simulation using 56 cores
   2022-06-29 16:55:00, completed in 5.31 seconds
   2022-06-29 16:55:03, e2e simulation using 56 cores
   2022-06-29 16:55:13, completed in 10.21 seconds
mode = RAVC, leakage = 2.36e-02
   2022-06-29 16:55:15, e2e simulation using 56 cores
   2022-06-29 16:55:19, completed in 4.71 seconds
   2022-06-29 16:55:22, e2e simulation using 56 cores
   2022-06-29 16:55:27, completed in 4.63 seconds
   2022-06-29 16:55:29, e2e simulation using 56 cores
   2022-06-29 16:55:41, completed in 11.36 seconds
mode = CVC, leakage = 4.71e-02
   2022-06-29 16:55:43, e2e simulation using 56 cores
   2022-06-29 16:55:53, completed in 10.19 seconds


### long exposure

In [5]:
conf = dict(
    dir_output = 'instant_psf/long_exp',
    nframes_avg = 100,
    add_phase = True,
)
get_psfs(conf)

   2022-06-29 16:55:55, e2e simulation using 56 cores
   2022-06-29 16:56:26, completed in 31.39 seconds
ELT peak = 0.0118
   2022-06-29 16:56:30, e2e simulation using 56 cores
   2022-06-29 16:57:31, completed in 61.29 seconds
   2022-06-29 16:57:36, e2e simulation using 56 cores
   2022-06-29 16:58:56, completed in 80.27 seconds
mode = RAVC, leakage = 2.43e-02
   2022-06-29 16:58:59, e2e simulation using 56 cores
   2022-06-29 16:59:59, completed in 59.27 seconds
   2022-06-29 17:00:03, e2e simulation using 56 cores
   2022-06-29 17:01:03, completed in 60.43 seconds
   2022-06-29 17:01:08, e2e simulation using 56 cores
   2022-06-29 17:02:32, completed in 84.9 seconds
mode = CVC, leakage = 4.74e-02
   2022-06-29 17:02:37, e2e simulation using 56 cores
   2022-06-29 17:03:58, completed in 81.11 seconds
